In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
!pip install tmdbsimple
import tmdbsimple as tmdb
import os, time,json

# Part 1 

# - Importing the Data

In [5]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
# reading given tsv file
basics = pd.read_csv(basics_url, sep='\t', low_memory=True)

IncompleteRead: IncompleteRead(174435581 bytes read, 124578 more expected)

In [6]:
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=True)

IncompleteRead: IncompleteRead(6563281 bytes read, 108983 more expected)

In [7]:
akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'
df_reader = pd.read_csv(akas_url, sep='\t', low_memory=True, chunksize=100_000)
df_reader

IncompleteRead: IncompleteRead(308856267 bytes read, 525105 more expected)

In [ ]:
df_reader._currow

In [ ]:
temp_df = df_reader.get_chunk()
temp_df

In [ ]:
df_reader._currow

In [ ]:
temp_df.replace({'\\N':np.nan},inplace=True)
temp_df = temp_df[(temp_df['region'] == 'US')]
temp_df

In [ ]:
chunk_num=1
fname= f'Data/title_akas_chunk_{chunk_num:03d}.csv.gz'
fname

In [ ]:
## Save temp_df to disk using the fname.
temp_df.to_csv(fname, compression='gzip')
## incrementing chunk_num by 1 for the next file.
chunk_num+=1

In [ ]:
pd.read_csv(fname)

In [ ]:
 
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
chunk_num = 1
df_reader = pd.read_csv(akas_url, sep='\t',
                        low_memory=False, chunksize=100_000)

In [ ]:
for temp_df in df_reader:
        #### COMBINED WORKFLOW FROM ABOVE
    ## Replace "\N" with np.nan
    temp_df.replace({'\\N':np.nan},inplace=True)
    temp_df = temp_df[(temp_df['region'] == 'US')]
     
    ### Saving chunk to disk
    fname= f'title_akas_chunk_{chunk_num:03d}.csv.gz'
    temp_df.to_csv(fname, compression='gzip')
    print(f"- Saved {fname}")
    
    ## increment chunk_num    
    chunk_num+=1
## Closing the reader now that we are done looping through the file
df_reader.close()

In [ ]:
import glob
q = "title_akas_chunk*.csv.gz"
chunked_files = glob.glob(q)
# Showing the first 5
chunked_files[:5]

In [ ]:
q = "title_akas_chunk*.csv.gz"
chunked_files = sorted(glob.glob(q))
# Showing the first 5 
chunked_files[:5]

In [ ]:
# Recursive query - extra /**/ added to string
q = "**/title_akas_chunk*.csv.gz" 
chunked_files = sorted(glob.glob(q, recursive=True)) 
# Showing the first 5 
chunked_files[:5]

In [ ]:
## Loading all files as df and appending to a list
df_list = []
for file in chunked_files:
    temp_df = pd.read_csv(file, index_col=0)
    df_list.append(temp_df)
    
## Concatenating the list of dfs into 1 combined
df_combined = pd.concat(df_list)
df_combined

In [ ]:
## Saving the final combined dataframe
final_fname ='title_akas_combined.csv.gz'
df_combined.to_csv(final_fname, compression='gzip', index=False)

# Filtering/ Cleaning
1. Title Basics

In [ ]:
basics.replace({'\\N':np.nan},inplace=True)

In [ ]:
basics.head()

In [ ]:
basics = basics.dropna(subset=["runtimeMinutes"])

In [ ]:
basics = basics.dropna(subset=["genres"])

In [ ]:
basics = basics.loc[(basics['titleType']=='movies')] 
basics

In [ ]:
basics = basics[(basics['startYear']>=2000)&(basics['startYear']<2022)]
basics

In [ ]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics_df = basics[~is_documentary]


In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(basics['titleType'])
keepers

In [ ]:
basics = basics[keepers]
basics

In [ ]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [ ]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()